In [ ]:
from Functions_1 import normalize_data, time_series_CV_split, wrapper_feature_selector, train_and_predict, warn
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.preprocessing import MinMaxScaler 
from time import time
from math import sqrt
from sklearn.linear_model import BayesianRidge
from matplotlib import pyplot
import plotly.offline as py
import plotly.graph_objs as go
import warnings
warnings.warn = warn
py.init_notebook_mode(connected=True)
%matplotlib inline

# Train Set
train_data = pd.read_csv('/Users/Andy/Desktop/Bitcoin/Data/Train_data_after_EDA.csv')
print('n_features:', len(train_data.iloc[0]))
print('n_samples:', len(train_data))
train_data.head()

In [ ]:
# Test Set
test_data = pd.read_csv('/Users/Andy/Desktop/Bitcoin/Data/Test_data_after_EDA.csv')
print('n_features:', len(test_data.iloc[0]))
print('n_samples:', len(test_data))
test_data.head()

In [ ]:
combined_data = pd.concat([train_data, test_data], ignore_index=True) # use data from 01/2018 onwards only - best stationarity
combined_data['Price'] = combined_data['close'].shift(-1) # Dependent variable Y

In [ ]:
combined_data

In [ ]:
# Combined train and test sets
combined_data = pd.concat([train_data, test_data], ignore_index=True) # use data from 01/2018 onwards only - best stationarity
combined_data['Price'] = combined_data['close'].shift(-1) # Dependent variable Y
print('n_features:', len(combined_data.iloc[0]))
print('n_samples:', len(combined_data))
combined_data = combined_data.iloc[:-1,1:]
combined_data.tail()

In [ ]:
# Feature Meta Subset
subset = [8, 7, 6, 5, 4, 3, 2, 1, 0] 

# Split train data into X (features) and Y (dependent variable)
data = combined_data.values 
Y_train = data[:-181,-1].reshape(-1,1) 
X_train = data[:-181,:-1]

# Training Validation samples size (1/10/18 - 31/12/18)
n_validation = 92

# Feature Selection
selected_features = wrapper_feature_selector(X_train,Y_train,BayesianRidge(),subset)[0]   
print('Selected Features:',str(selected_features))


In [ ]:
%%time
rmse,Y_train_test,Y_train_pred= train_and_predict(X_train[:,selected_features],Y_train,BayesianRidge(),predict=False)
print('Train RMSE: {:0.2f}'.format(rmse))
        

In [ ]:
%%time
# Feature Meta Subset
subset = [8, 7, 6, 5, 4, 3, 2, 1, 0] 

# Split train data into X (features) and Y (dependent variable)
data = combined_data.values 
Y_train = data[:-181,-1].reshape(-1,1) 
X_train = data[:-181,:-1]

# Test samples size (1/1/19 - 30/6/19)
n_validation = 179

# Split test data into X (features) and Y (dependent variable)
Y_test = data[:,-1].reshape(-1,1) # including train data for fitting the model
X_test = data[:,:-1]

rmse,Y_test,Y_pred= train_and_predict(X_test,Y_test,BayesianRidge(),n_validation,subset,predict=True)
print('Test RMSE: {:0.2f}'.format(rmse))


In [ ]:
# Output prediction to csv file
output = {'Y_pred': Y_pred.reshape(-1,)}
output = DataFrame(output)
output.to_csv('BayesianRidge.csv', index=False)


In [ ]:
trace1 = go.Scatter(
    x = np.arange(0, len(Y_pred), 1),
    y = Y_pred.reshape(-1,),
    mode = 'lines',
    name = 'Predicted labels',
    line = dict(color=('rgb(244, 146, 65)'), width=2)
)
trace2 = go.Scatter(
    x = np.arange(0, len(Y_test), 1),
    y = Y_test.reshape(-1,),
    mode = 'lines',
    name = 'True labels',
    line = dict(color=('rgb(66, 244, 155)'), width=2)
)

layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our model predicted',
             xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=[trace1, trace2], layout=layout)
py.iplot(fig, filename='results_demonstrating0')

